In [1]:
import os
import time
import arxiv
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings

C:\Users\aanch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Create directory if not exists
dirpath = "arxiv_papers"
if not os.path.exists(dirpath):
    os.makedirs(dirpath)

In [3]:
# Search arXiv for papers related to "LLM"
client = arxiv.Client()
search = arxiv.Search(
    query="LLM",
    max_results=10,
    sort_order=arxiv.SortOrder.Descending
)

In [4]:
# Download and save the papers
for result in client.results(search):
    while True:
        try:
            result.download_pdf(dirpath=dirpath)
            print(f"-> Paper id {result.get_short_id()} with title '{result.title}' is downloaded.")
            break
        except (FileNotFoundError, ConnectionResetError) as e:
            print("Error occurred:", e)
            time.sleep(5)

-> Paper id 2406.10300v1 with title 'Large Language Models as Software Components: A Taxonomy for LLM-Integrated Applications' is downloaded.
-> Paper id 2405.19888v1 with title 'Parrot: Efficient Serving of LLM-based Applications with Semantic Variable' is downloaded.
-> Paper id 2311.10372v2 with title 'A Survey of Large Language Models for Code: Evolution, Benchmarking, and Future Trends' is downloaded.
-> Paper id 2404.14809v1 with title 'A Survey of Large Language Models on Generative Graph Analytics: Query, Learning, and Applications' is downloaded.
-> Paper id 2308.08241v2 with title 'TEST: Text Prototype Aligned Embedding to Activate LLM's Ability for Time Series' is downloaded.
-> Paper id 2408.02479v1 with title 'From LLMs to LLM-based Agents for Software Engineering: A Survey of Current, Challenges and Future' is downloaded.
-> Paper id 2402.18050v1 with title 'MEGAnno+: A Human-LLM Collaborative Annotation System' is downloaded.
-> Paper id 2402.08030v1 with title 'Why and 

In [5]:
# Load papers from the directory
papers = []
loader = DirectoryLoader(dirpath, glob="./*.pdf", loader_cls=PyPDFLoader)
try:
    papers = loader.load()
except Exception as e:
    print(f"Error loading file: {e}")
print("Total number of pages loaded:", len(papers)) 

Total number of pages loaded: 239


In [6]:
# Concatenate all pages' content into a single string
full_text = ''
for paper in papers:
    full_text += paper.page_content

In [7]:
# Remove empty lines and join lines into a single string
full_text = " ".join(line for line in full_text.splitlines() if line)
print("Total characters in the concatenated text:", len(full_text)) 

Total characters in the concatenated text: 1056041


In [8]:
# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
paper_chunks = text_splitter.create_documents([full_text])

In [16]:
from qdrant_client import QdrantClient

client = QdrantClient("localhost", port=6333)

In [9]:
# Create Qdrant vector store
qdrant = Qdrant.from_documents(
    documents=paper_chunks,
    embedding=GPT4AllEmbeddings(),
    path="./tmp/local_qdrant",
    collection_name="arxiv_papers",
)
retriever = qdrant.as_retriever()

In [10]:
# Define prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
# Initialize Ollama LLM
ollama_llm = "llama2:7b-chat"
model = ChatOllama(model=ollama_llm)

C:\Users\aanch\AppData\Local\Temp\ipykernel_3668\2757140565.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model=ollama_llm)


In [12]:
# Define the processing chain
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
# Add typing for input
class Question(BaseModel):
    __root__: str

In [14]:

# Apply input type to the chain
chain = chain.with_types(input_type=Question)
result = chain.invoke("Explain about Vision Enhancing LLMs")
print(result)

Based on the provided context, Vision Enhancing LLMs (VELLMs) are a type of Language Model that can enhance or augment visual input with text-based content. The main difference between VELLMs and traditional LLMs is their ability to integrate with other systems and modalities, such as visual input or real-world perception data, enabling them to perform more complex and context-based decision-making tasks.

VELLMs are designed to process and generate text-based content in multi-modal settings, where they can interact with visual inputs or real-world data to improve their performance. This allows VELLMs to perform tasks that traditional LLMs cannot, such as understanding the context of visual inputs or generating responses based on both textual and visual inputs.

The key insight of this research is that rather than analyzing an LLM-integrated application in whole, analysis should start with the identification and description of its distinct LLM components. This allows for a clearer unde